In [1]:
from glob import glob 
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib

In [2]:
!pwd

/home/gowreesh/Desktop/WUINtern/Dataset


In [3]:
!ls

chromedriver_linux64  chromedriver_linux64.zip	DownloadImages.ipynb  images


In [4]:
glob('/chromedriver_linux64/chromedriver')

[]

In [5]:
DRIVER_PATH = 'chromedriver_linux64/chromedriver'

In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=0.2):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(5)
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [7]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [8]:
IMAGES_PATH = 'images/timing belt'

In [9]:
wd = webdriver.Chrome(executable_path=DRIVER_PATH)
queries = ["timing belt car part"]  #change your set of querries here
for query in queries:
    wd.get('https://google.com')
    search_box = wd.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(query)
    links = fetch_image_urls(query,400,wd)
    #images_path = '/Users/anand/Desktop/contri/images'  #enter your desired image path
    images_path = IMAGES_PATH+'/'
    if links is not None:
        for i in links:
            persist_image(images_path,query,i)
wd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 80 image links, looking for more ...
Found: 100 search results. Extracting links from 100:100
Found: 80 image links, looking for more ...
Found: 100 search results. Extracting links from 100:100
Found: 80 image links, looking for more ...
Found: 100 search results. Extracting links from 100:100
Found: 80 image links, looking for more ...
Found: 100 search results. Extracting links from 100:100
Found: 80 image links, looking for more ...
Found: 100 search results. Extracting links from 100:100
Found: 80 image links, looking for more ...
Found: 212 search results. Extracting links from 100:212
Found: 182 image links, looking for more ...
Found: 300 search results. Extracting links from 212:300
Found: 275 image links, looking for more ...
Found: 400 search results. Extracting links from 300:400
Found: 383 image links, looking for more ...
Found: 400 search results. Extracting links from 400:400
Found: 383 image links, looking f

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQZkWWoBuNXbBDxSl3lTRY0LhiQ_4HzRa9-fg&usqp=CAU - as images/timing belt/timing belt car part/a5131fd052.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRSo-ygq74Iyq07lF-hk67pkEhlFlv3iP_eVA&usqp=CAU - as images/timing belt/timing belt car part/0449aed216.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT93zcZUr9jAXyTZoDwK6brym0WDqkN0t3Qeg&usqp=CAU - as images/timing belt/timing belt car part/5e29fe421c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSYa4evxPUKg1GRM34qz3tqXk7WXqGwHncCUg&usqp=CAU - as images/timing belt/timing belt car part/f170db177c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTarFa28bvQ_syxCzRaQBqRgxor3b4cPQuVxQ&usqp=CAU - as images/timing belt/timing belt car part/60c7c6bcab.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTT1ofiyq-Oo7f2NJm8CwKyWUmylWkvvIAVIw&u

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSNOCF4Cpc7FlYnnimfsZ66Liu2vyUOpFNfnw&usqp=CAU - as images/timing belt/timing belt car part/e53ff4aac8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSIVXwwRLhIuzgCjPy8xkUuBNsGLaIyJcpxbQ&usqp=CAU - as images/timing belt/timing belt car part/b1186e36f3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS0AP8jwY7Gzu3lcdl8d1np6d3R1s8Tp-MkPQ&usqp=CAU - as images/timing belt/timing belt car part/9751a2e96f.jpg
SUCCESS - saved https://www.gazzellaracing.com/var/images/product/300.300/4191014WP.jpg - as images/timing belt/timing belt car part/5f52d30339.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTs83z45OZILPZtrbV5uZDggPGCnUFduhBGKg&usqp=CAU - as images/timing belt/timing belt car part/5600a35b3a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRXdFhpW8cY5bpp7b572FD4gmPBwNMPtQjUOQ&usqp=CAU - as images/timing bel

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTE9UTh0rI1oNYa1m2miZXGKemX_3yJRETPrQ&usqp=CAU - as images/timing belt/timing belt car part/66bdc15cd4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQVWRL3WatZoz37GeeG8bt2Ro0va57-cWeHTQ&usqp=CAU - as images/timing belt/timing belt car part/fe17f7b1f2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR3kb9MU4WiMMQ2Bm51CUySRtdS1rrS5kzPSg&usqp=CAU - as images/timing belt/timing belt car part/b959195316.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTsY8KJDsQuGLeQIy4Jn7ThOOntqEwdvZctgg&usqp=CAU - as images/timing belt/timing belt car part/e4e465c741.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQhEgvR6D6G8qWea61NYFj7dCjJaj66TlGskQ&usqp=CAU - as images/timing belt/timing belt car part/7b15179136.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS6kBsr3Ba6L5FV4oPA03zCNeWesaSWvEO1FA&u

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQqrRFFOkZlXkfQGPSeK1A33d_ZqfV8LLsrdA&usqp=CAU - as images/timing belt/timing belt car part/0bdd59240c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQy6qfLS5fYerQX4vyLWAtSnfMOWF-x-IBD7Q&usqp=CAU - as images/timing belt/timing belt car part/c8ff718ba2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRPiC4acSksGXJc3H3VTT01Z0qIafLzFCPgrQ&usqp=CAU - as images/timing belt/timing belt car part/2f2d2a7bd8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTZJBEFKYgfWgrbni18hp7L18Hse2iZfxOL9g&usqp=CAU - as images/timing belt/timing belt car part/10b954f372.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQYjexFuDmK-X7WP9t79ZvQcDxJmZ8SCNIv-A&usqp=CAU - as images/timing belt/timing belt car part/6b934926d1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQSdRw4jBCDTjl0Zgf0yqZXQIC-5XuO6ZQebw&u

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQQNLFoLcrXmbIuOyY1dzMsIwzdbNGJCdlrhQ&usqp=CAU - as images/timing belt/timing belt car part/b73663dc88.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS2qO1JRk7Tv1MRajZ4Xk_O_5aQOApNd0qJqA&usqp=CAU - as images/timing belt/timing belt car part/ca909e9d09.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSfJrhiTRPo-V_QpC5YmV6Ovg8ywRbzX_uQ8A&usqp=CAU - as images/timing belt/timing belt car part/3d2dbccc4a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQadlqaAP-ZYy1CDYSa3gc6DIjIJtN6FcHIiQ&usqp=CAU - as images/timing belt/timing belt car part/abdae34de9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRpWsnsHG5hFL3v3nWc6Wu8WC4XEfIYMTgvkw&usqp=CAU - as images/timing belt/timing belt car part/d0c01f1a0a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTq2TrXeCqHDhPWAcw_WK2Vau4dpU4Hl7i9rA&u

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQP0cNL--PXHcPogDZXiNz36qXjvvmOVYLLrA&usqp=CAU - as images/timing belt/timing belt car part/df2505e582.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcThZ6SkvNQFPLVuFd0Pszcbv-SgxTso60vPSw&usqp=CAU - as images/timing belt/timing belt car part/6abbfbc829.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQUf1rnr_ld5gnQQGrdAz69Ih3lJiTV7ZbtfA&usqp=CAU - as images/timing belt/timing belt car part/06ceb4a610.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRoTYR6HaVgwhcdpMnqs-HRxdDncuECxRKkNA&usqp=CAU - as images/timing belt/timing belt car part/87bb748f40.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQANONDwHlOAlH3IqR7swUyD6IeMqHeJHsA6A&usqp=CAU - as images/timing belt/timing belt car part/e76ca6c446.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTEj6opPWzXYKJXozLPkNgnRGEbn0t00ZCYQQ&u

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTh8i3ZHJrkLW-KU0FP2aAOWByn7jX4T8llGw&usqp=CAU - as images/timing belt/timing belt car part/d28f27ed5c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQQZW0r-CRgZ1d8_m9Brtch0dt4oM6tdFYLgg&usqp=CAU - as images/timing belt/timing belt car part/5dfd7f1df9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQMNHjEI-0KISUCFSoLTBu9207P82scYyof9w&usqp=CAU - as images/timing belt/timing belt car part/4da88b7f3a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQEc0-8GBi8ozioyjAhNTxYpQKUkQj_K09Wzw&usqp=CAU - as images/timing belt/timing belt car part/ec9e5ef09f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT3g3xkJk2d5CJzhRu8KIqy7Qkky0QVhpQdZg&usqp=CAU - as images/timing belt/timing belt car part/99bf52e996.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS6J3twx8V0Z_guxukuGdBB71SklWvlinVXDw&u

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSXdJrXPRwHVzl1ctRQn1kG86zkGj8io1pyuw&usqp=CAU - as images/timing belt/timing belt car part/ed45d2bf03.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTDkre_GGQCBh5CRQy9qhRqGXWtF6lzeHG5mw&usqp=CAU - as images/timing belt/timing belt car part/ab5b87261c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTZtts0e7haFR9Cq5sIBISvaqAw7N0wRp0JkQ&usqp=CAU - as images/timing belt/timing belt car part/aaefb95dff.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSYfjgRcAtQPh-CmBydMQR_n2r_FhHLCp-8iQ&usqp=CAU - as images/timing belt/timing belt car part/408f8ddf29.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSpVOXKvc4G1UhVakSGKD44Rw9iPPrMRqtkjQ&usqp=CAU - as images/timing belt/timing belt car part/6f22798ca0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTGtkSASfWlKVRWtvNJou4fnK2XWv2h41_YGQ&u